# scrape ST links and articles by year

In [1]:
# scraping year by year
OUTPUT_XLSX = "Output/ST/st_articles_2024_Jan_to_Dec.xlsx"
CHECKPOINT_CSV = "Output/ST/st_articles_2024_Jan_to_Dec_checkpoint.csv"

In [2]:
# DATE PARAMETERS
MONTHS = ["01","02","03","04","05","06","07", "08", "09", "10", "11", "12"]
SITEMAP_TMPL = "https://www.straitstimes.com/sitemap/2024/{m}/feeds.xml"


In [3]:
# WEBSITES TO SCRAPE FROM
# label: how it will appear in the new 'website' column
# base_url: the listing page (to exclude if it ever shows up)
# match_substrings: if any of these strings is in the URL, we treat it as belonging to this website
WEBSITE_SOURCES = [
    {
        "label": "housing",
        "base_url": "https://www.straitstimes.com/singapore/housing",
        "match_substrings": ["/singapore/housing/"],
    },
    {
        "label": "tag_hdb",
        "base_url": "https://www.straitstimes.com/tags/hdb",
        "match_substrings": ["/hdb"],
    },
    {
        "label": "tag_public_housing",
        "base_url": "https://www.straitstimes.com/tags/public-housing",
        "match_substrings": ["/public-housing"],
    },
    {
        "label": "tag_singapore_property",
        "base_url": "https://www.straitstimes.com/tags/singapore-property",
        "match_substrings": ["/singapore-property"],
    },
    {
        "label": "tag_residential_property",
        "base_url": "http://straitstimes.com/tags/residential-property",
        "match_substrings": ["/residential-property"],
    },
    {
        "label": "tag_private_property",
        "base_url": "https://www.straitstimes.com/tags/private-property",
        "match_substrings": ["/private-property"],
    },
]

In [4]:
# libraries
import warnings
warnings.filterwarnings("ignore")

from __future__ import annotations

import os
import csv
import re
import time
from datetime import date

import pandas as pd
import pytz # timezone handling
import requests
from bs4 import BeautifulSoup
from dateutil import parser as dtparser # flexibly parse date/time strings into datetime objects
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


In [5]:
REQUEST_TIMEOUT = 30
SLEEP_BETWEEN_ARTICLES = 0.3  # politeness delay (seconds) between article hits
STOP_PHRASES = [
    # e.g. "Join ST's Telegram channel",
    #      "Get unlimited access",
]

# URL to open for manual login (home page or dedicated login page)
LOGIN_URL = "https://www.straitstimes.com/"  # you can change to a specific login URL if you prefer

# =================================

TZ_SGT = pytz.timezone("Asia/Singapore")

HEADERS = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/120.0 Safari/537.36"
    )
}

PUBLISHED_RE = re.compile(
    r"^Published\s+[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}"
    r"(?:,\s+\d{1,2}:\d{2}\s*(?:AM|PM))?\s*$",
    re.IGNORECASE,
)
UPDATED_RE = re.compile(
    r"^Updated\s+[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}"
    r"(?:,\s+\d{1,2}:\d{2}\s*(?:AM|PM))?\s*$",
    re.IGNORECASE,
)


# ========== SESSION FOR SITEMAPS (REQUESTS) ==========

def get_sitemap(url: str) -> str:
    """Fetch sitemap XML using requests (no login needed)."""
    r = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
    if not r.encoding or r.encoding.lower() == "iso-8859-1":
        r.encoding = "utf-8"
    r.raise_for_status()
    return r.text


# ========== BROWSER FOR LOGGED-IN ARTICLE PAGES (SELENIUM) ==========

def create_logged_in_driver() -> webdriver.Chrome:
    """
    Open a real browser window, let the user log in manually,
    and wait for them to press Enter before scraping.
    """
    options = Options()
    # If you want headless, uncomment:
    # options.add_argument("--headless=new")

    driver = webdriver.Chrome(options=options)
    driver.get(LOGIN_URL)
    print(
        "A browser window has opened.\n"
        "1) Please log in to The Straits Times with your subscription in that window.\n"
        "2) Once you see that you're logged in (e.g. access to full articles),\n"
        "   come back to this terminal and press Enter to start scraping."
    )
    input("After you have successfully logged in, press Enter here to continue... ")
    return driver


# ========== URL CLASSIFICATION ==========

def classify_website(url: str):
    """
    Return a label (from WEBSITE_SOURCES) if this URL matches any
    configured website patterns; otherwise return None.
    """
    u_strip = url.rstrip("/")
    for src in WEBSITE_SOURCES:
        # Skip the pure listing page itself, if it ever appears
        if u_strip == src["base_url"].rstrip("/"):
            return None
        for pat in src["match_substrings"]:
            if pat in url:
                return src["label"]
    return None


# ========== TEXT / CONTENT HELPERS ==========

def clean_text(txt: str) -> str:
    txt = txt.replace("\u00a0", " ")
    txt = re.sub(r"[ \t]+", " ", txt)       # collapse multiple spaces
    txt = re.sub(r"\n{3,}", "\n\n", txt)    # collapse >2 blank lines
    return txt.strip()


def cut_at_stop_phrases(text: str, stops) -> str:
    if not stops:
        return text
    lower = text.lower()
    cut_idx = None
    for phrase in stops:
        i = lower.find(phrase.lower())
        if i != -1:
            cut_idx = i if cut_idx is None else min(cut_idx, i)
    return text[:cut_idx].rstrip() if cut_idx is not None else text


def is_promo_line(t: str) -> bool:
    t_low = t.lower()
    return ("sign up now" in t_low and "newsletter" in t_low) or (
        "newsletters delivered to your inbox" in t_low
    )


def is_photo_credit(t: str) -> bool:
    t_strip = t.strip()
    return t_strip.upper().startswith("PHOTO:") or t_strip.upper().startswith("ST PHOTO:")


def is_byline(t: str) -> bool:
    # Common byline patterns: "By John Doe", "By JOHN DOE", or just a short proper-name line
    if t.lower().startswith("by "):
        return True
    tokens = t.strip().split()
    if 1 <= len(tokens) <= 4:
        cap_like = 0
        for w in tokens:
            if re.match(r"^[A-Z][a-z]+(-[A-Z][a-z]+)?$", w) or re.match(r"^[A-Z]{2,}$", w):
                cap_like += 1
        if cap_like == len(tokens):
            return True
    return False


def find_content_paragraphs(soup: BeautifulSoup):
    """
    Heuristically locate the paragraphs that comprise the article body.
    Returns a list of elements (usually <p> or <div>) in document order.

    Special handling for Straits Times:
    - Prefer content inside .storyline-wrapper / .archival-wrapper
    - Ignore paragraphs nested inside <a> (to avoid duplicate link text)
    - Ignore photo credits and promo lines
    """

    # 1. Remove obvious non-content elements, including images
    for sel in [
        "script",
        "style",
        "noscript",
        "header",
        "footer",
        "form",
        "aside",
        "figure",
        "picture",
        "img",
        "figcaption",
    ]:
        for tag in soup.select(sel):
            tag.decompose()

    # 2. Straits Times specific containers: storyline + archival-wrapper
    st_containers = soup.select("div.storyline-wrapper.default, div.storyline-wrapper")
    st_paragraph_nodes = []

    for cont in st_containers:
        # (a) Chuan Park style: text inside <div class="archival-wrapper"><span>...</span></div>
        for block in cont.select("div.archival-wrapper"):
            txt = block.get_text(" ", strip=True)
            if txt:
                st_paragraph_nodes.append(block)

        # (b) Usual <p> paragraphs for newer templates
        for p in cont.find_all("p"):
            # Skip paragraphs that are themselves inside a link – we will
            # keep only the outer paragraph so we don't double-count link text.
            if p.find_parent("a"):
                continue

            txt = p.get_text(" ", strip=True)
            if not txt:
                continue
            if is_photo_credit(txt) or is_promo_line(txt):
                continue
            st_paragraph_nodes.append(p)

    # If we found a reasonable number of paragraphs in storyline-wrapper, use them
    if len(st_paragraph_nodes) >= 3:
        return st_paragraph_nodes

    # 3. Generic fallback for other sites / older templates

    selectors = [
        '[itemprop="articleBody"]',
        '[property="articleBody"]',
        "article",
        ".article-body",
        ".article__content",
        ".content-body",
        ".rich-text",
        ".field--name-body",
        ".c-article-content",
        ".article-content",
        ".story-content",
        ".field-item",
    ]
    candidates = []
    for sel in selectors:
        nodes = soup.select(sel)
        if nodes:
            candidates.extend(nodes)

    if not candidates:
        main = soup.find("main")
        candidates = [main] if main else [soup]

    best_ps = []
    for node in candidates:
        ps = []
        for p in node.find_all("p"):
            if p.find_parent("a"):
                continue
            txt = p.get_text(" ", strip=True)
            if not txt:
                continue
            if is_photo_credit(txt) or is_promo_line(txt):
                continue
            ps.append(p)

        if len(ps) >= 3:
            best_ps = ps
            break
        elif ps and not best_ps:
            best_ps = ps

    # Last resort: all <p> on the page, with light filtering
    if not best_ps:
        ps = []
        for p in soup.find_all("p"):
            if p.find_parent("a"):
                continue
            txt = p.get_text(" ", strip=True)
            if not txt:
                continue
            if is_photo_credit(txt) or is_promo_line(txt):
                continue
            ps.append(p)
        best_ps = ps

    return best_ps


def extract_article_content_with_lead(soup: BeautifulSoup) -> str:
    """
    Keep one 'lead line' between the promo ('sign up now ... newsletters') and the 'Published ...' header,
    skipping photo credits and bylines. Then include everything after Published (and optional Updated).
    """
    ps = find_content_paragraphs(soup)
    if not ps:
        return ""

    # Normalized paragraph texts
    p_texts = [clean_text(p.get_text(" ", strip=True)) for p in ps]

    # Locate indices of interest
    promo_idx = None
    published_idx = None
    for i, t in enumerate(p_texts):
        if promo_idx is None and is_promo_line(t):
            promo_idx = i
        if published_idx is None and (PUBLISHED_RE.match(t) or t.startswith("Published ")):
            published_idx = i
            break

    # If no 'Published' header, fallback to entire body
    if published_idx is None:
        content = clean_text("\n\n".join(p_texts))
        return cut_at_stop_phrases(content, STOP_PHRASES)

    # Choose lead line: scan from (promo_idx+1 if found else max(published_idx-6,0))
    lead_line = ""
    scan_start = (promo_idx + 1) if promo_idx is not None else max(published_idx - 6, 0)
    for j in range(scan_start, published_idx):
        cand = p_texts[j]
        if not cand:
            continue
        if is_photo_credit(cand) or is_byline(cand) or is_promo_line(cand):
            continue
        # Prefer a substantive sentence (ends with .!? or length threshold)
        if len(cand) >= 40 or re.search(r"[.!?]$", cand):
            lead_line = cand
            break

    # Skip 'Published ...' and optional 'Updated ...'
    start_idx = published_idx + 1
    if start_idx < len(p_texts) and (
        UPDATED_RE.match(p_texts[start_idx]) or p_texts[start_idx].startswith("Updated ")
    ):
        start_idx += 1

    # Join parts
    parts = []
    if lead_line:
        parts.append(lead_line)
    parts.extend(p_texts[start_idx:])

    content = clean_text("\n\n".join(parts))
    content = cut_at_stop_phrases(content, STOP_PHRASES)

    # Fallback if empty
    if not content:
        content = clean_text("\n\n".join(p_texts))
        content = cut_at_stop_phrases(content, STOP_PHRASES)

    return content


# ========== ARTICLE PARSING (METADATA + CONTENT, USING DRIVER) ==========

def parse_article(url: str, driver: webdriver.Chrome):
    """
    Use the logged-in Selenium browser to fetch an article URL once
    and return:
    {
        "title": ...,
        "url": ...,
        "published_date": ...,
        "content": ...,
        "char_count": ...
    }
    or None if parsing fails.
    """
    driver.get(url)
    # If the page is JS-heavy, you can wait a bit or add explicit waits here
    time.sleep(1.5)  # adjust if needed

    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    # ---- Title ----
    title = (soup.find("meta", {"property": "og:title"}) or {}).get("content")
    if not title:
        h1 = soup.find("h1")
        title = h1.get_text(" ", strip=True) if h1 else None

    # ---- Published time ----
    pub = None
    for tag, attrs in [
        ("meta", {"property": "article:published_time"}),
        ("meta", {"name": "article:published_time"}),
        ("meta", {"itemprop": "datePublished"}),
        ("meta", {"name": "pubdate"}),
    ]:
        m = soup.find(tag, attrs=attrs)
        if m and m.get("content"):
            pub = m["content"].strip()
            break

    if not pub:
        t = soup.find("time")
        if t and (t.get("datetime") or t.get("content")):
            pub = (t.get("datetime") or t.get("content")).strip()

    if not pub:
        m = re.search(
            r"Published\s+[A-Za-z]{3,9}\s+\d{1,2},\s+\d{4}"
            r"(?:,\s+\d{1,2}:\d{2}\s*(?:AM|PM))?",
            soup.get_text("\n", strip=True),
        )
        pub = m.group(0).replace("Published", "").strip() if m else None

    if not title or not pub:
        return None

    # ---- Parse datetime and convert to SGT ----
    dt = dtparser.parse(pub)
    dt = TZ_SGT.localize(dt) if dt.tzinfo is None else dt.astimezone(TZ_SGT)

    # ---- Extract content ----
    content = extract_article_content_with_lead(soup)
    char_count = len(content)

    return {
        "title": title,
        "url": url,
        "published_date": dt.isoformat(),  # renamed from published_sgt
        "content": content,
        "char_count": char_count,
    }


# ========== CHECKPOINT HELPER ==========

def append_checkpoint_row(row_dict: dict, checkpoint_path: str):
    """
    Append a single article row to the checkpoint CSV.
    Creates the file with header if it does not exist yet.
    """
    fieldnames = ["website", "title", "url", "published_date", "content", "char_count"]

    file_exists = os.path.exists(checkpoint_path)

    with open(checkpoint_path, "a", newline="", encoding="utf-8-sig") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        out_row = {k: row_dict.get(k, "") for k in fieldnames}
        writer.writerow(out_row)


# ========== MAIN WORKFLOW (SITEMAPS → CHECKPOINT → EXCEL) ==========

def main():
    # 0. Open browser and let user log in manually
    driver = create_logged_in_driver()

    try:
        # 1. Collect all matching URLs + website labels from sitemaps
        url_records = []  # list of (url, website_label)
        for m in MONTHS:
            print(f"Fetching sitemap for month {m}...")
            xml = get_sitemap(SITEMAP_TMPL.format(m=m))
            sx = BeautifulSoup(xml, "xml")
            for loc in sx.find_all("loc"):
                u = loc.get_text(strip=True)
                website_label = classify_website(u)
                if website_label:
                    url_records.append((u, website_label))

        # 2. Deduplicate by URL (keep first label encountered)
        seen = {}
        unique_records = []
        for u, label in url_records:
            if u not in seen:
                seen[u] = label
                unique_records.append((u, label))

        print(f"Found {len(unique_records)} URLs for configured websites.")

        # 3. Load checkpoint (if exists) to know which URLs are already done
        done_urls = set()
        if os.path.exists(CHECKPOINT_CSV):
            print(f"Found existing checkpoint: {CHECKPOINT_CSV}")
            try:
                df_ckpt = pd.read_csv(CHECKPOINT_CSV, encoding="utf-8-sig")
                if "url" in df_ckpt.columns:
                    done_urls = set(df_ckpt["url"].dropna().astype(str))
                    print(f"{len(done_urls)} URLs already scraped (from checkpoint).")
            except Exception as e:
                print(f"WARNING: failed to read checkpoint ({e}). Proceeding as if none.")

        total = len(unique_records)

        # 4. For each URL, parse metadata + content using logged-in driver
        for i, (u, website_label) in enumerate(unique_records, 1):
            if u in done_urls:
                print(f"[{i}/{total}] SKIP (already in checkpoint) [{website_label}] - {u}")
                continue

            try:
                art = parse_article(u, driver=driver)
                if art:
                    art["website"] = website_label  # add website label

                    # Append immediately to checkpoint so we don't lose progress
                    append_checkpoint_row(art, CHECKPOINT_CSV)
                    done_urls.add(u)

                    status = f"OK (chars={art['char_count']})"
                else:
                    status = "SKIP (missing title/pubdate)"
            except Exception as e:
                status = f"ERR ({e})"
            print(f"[{i}/{total}] {status} [{website_label}] - {u}")
            time.sleep(SLEEP_BETWEEN_ARTICLES)

        # 5. Build final DataFrame from checkpoint, drop duplicates, and write Excel
        if os.path.exists(CHECKPOINT_CSV):
            df_out = pd.read_csv(CHECKPOINT_CSV, encoding="utf-8-sig")

            # Drop duplicates based on title + content
            before = len(df_out)
            df_out = df_out.drop_duplicates(subset=["title", "content"], keep="first")
            after = len(df_out)
            print(f"Removed {before - after} duplicate rows based on (title, content).")

            with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl") as writer:
                df_out.to_excel(writer, sheet_name="articles", index=False)

            print(f"Wrote {len(df_out)} rows to {OUTPUT_XLSX}")
        else:
            print("No checkpoint file found; nothing to write.")
    finally:
        # Always close the browser
        driver.quit()


if __name__ == "__main__":
    main()


A browser window has opened.
1) Please log in to The Straits Times with your subscription in that window.
2) Once you see that you're logged in (e.g. access to full articles),
   come back to this terminal and press Enter to start scraping.


After you have successfully logged in, press Enter here to continue...  


Fetching sitemap for month 01...
Fetching sitemap for month 02...
Fetching sitemap for month 03...
Fetching sitemap for month 04...
Fetching sitemap for month 05...
Fetching sitemap for month 06...
Fetching sitemap for month 07...
Fetching sitemap for month 08...
Fetching sitemap for month 09...
Fetching sitemap for month 10...
Fetching sitemap for month 11...
Fetching sitemap for month 12...
Found 147 URLs for configured websites.
[1/147] OK (chars=3750) [housing] - https://www.straitstimes.com/singapore/housing/tengah-residents-welcome-new-bus-stops-hope-for-more-bus-services-to-be-added
[2/147] OK (chars=4856) [housing] - https://www.straitstimes.com/singapore/housing/first-executive-condo-launch-of-2024-sells-53-of-units
[3/147] OK (chars=7454) [housing] - https://www.straitstimes.com/singapore/housing/2-former-schools-next-to-commonwealth-mrt-station-to-make-way-for-housing
[4/147] OK (chars=4036) [housing] - https://www.straitstimes.com/singapore/housing/new-homes-to-be-built-wit

# merge all scraped data files

In [103]:
import pandas as pd

df2020 = pd.read_excel("Output/ST/st_articles_2020_Jan_to_Dec.xlsx")
df2021 = pd.read_excel("Output/ST/st_articles_2021_Jan_to_Dec.xlsx")
df2022 = pd.read_excel("Output/ST/st_articles_2022_Jan_to_Dec.xlsx")
df2023 = pd.read_excel("Output/ST/st_articles_2023_Jan_to_Dec.xlsx")
df2024 = pd.read_excel("Output/ST/st_articles_2024_Jan_to_Dec.xlsx")
df2025 = pd.read_excel("Output/ST/st_articles_2025_Jan_to_Oct.xlsx")

In [2]:
len(df2020)

160

In [3]:
len(df2021)

210

In [4]:
len(df2022)

234

In [5]:
len(df2023)

142

In [6]:
len(df2024)

141

In [7]:
len(df2025)

110

In [104]:
# concat all
df = pd.concat([df2020, df2021, df2022, df2023, df2024, df2025], ignore_index=True)

In [105]:
# keep relevant columns
df.rename(columns={'char_count': 'content_chars'}, inplace=True)
df = df[['title', 'url', 'published_date', 'content', 'content_chars']]

In [106]:
# drop duplicates
df = df.drop_duplicates(keep='first').reset_index(drop=True)


In [107]:
# check if any duplicates
df['title'].duplicated().any()

True

In [108]:
# check if any duplicates
df['url'].duplicated().any()

False

In [109]:
len(df[df.duplicated(subset=['title'], keep=False)])

11

In [110]:
df[df.duplicated(subset=['title'], keep=False)]


title  \
183                      HDB upgraders help boost new condo prices in suburbs   
184                      HDB upgraders help boost new condo prices in suburbs   
225  More than 540,000 HDB flat owners to pay less in home insurance premiums   
226  More than 540,000 HDB flat owners to pay less in home insurance premiums   
378                       HDB resale market likely to remain robust this year   
380                       HDB resale market likely to remain robust this year   
391                               Upgrading works slated for 56,000 HDB flats   
393                               Upgrading works slated for 56,000 HDB flats   
395                               Upgrading works slated for 56,000 HDB flats   
457                                 HDB compensates BTO flat buyers for delay   
458                                 HDB compensates BTO flat buyers for delay   

                                                                                                                            url  \
183                         https://www.straitstimes.com/business/property/hdb-upgraders-help-boost-new-condo-prices-in-suburbs   
184                       https://www.straitstimes.com/business/property/hdb-upgraders-help-boost-new-condo-prices-in-suburbs-0   
225      https://www.straitstimes.com/singapore/housing/more-than-540000-hdb-flat-owners-to-pay-less-in-home-insurance-premiums   
226                 https://www.straitstimes.com/singapore/housing/540000-cpf-members-to-pay-less-for-home-protection-insurance   
378                 https://www.straitstimes.com/singapore/housing/analysis-hdb-resale-market-likely-to-remain-robust-this-year   
380                          https://www.straitstimes.com/singapore/housing/hdb-resale-market-likely-to-remain-robust-this-year   
391                                   https://www.straitstimes.com/singapore/housing/upgrading-works-slated-for-56000-hdb-flats   
393        https://www.straitstimes.com/singapore/housing/upgrading-works-start-for-56000-hdb-flats-built-between-1987-and-1997   
395                                  https://www.straitstimes.com/singapore/housing/upgrading-works-slated-for-56000-hdb-flats1   
457                                    https://www.straitstimes.com/singapore/housing/hdb-compensates-bto-flat-buyers-for-delay   
458  https://www.straitstimes.com/singapore/housing/compensation-for-buyers-of-sengkang-bto-flats-over-delay-in-completion-date   

                published_date  \
183  2021-02-14T05:00:00+08:00   
184  2021-02-14T05:00:00+08:00   
225  2021-06-25T05:00:00+08:00   
226  2021-06-24T12:20:39+08:00   
378  2022-01-15T05:00:00+08:00   
380  2022-01-15T05:00:00+08:00   
391  2022-02-20T21:00:00+08:00   
393  2022-02-20T13:00:00+08:00   
395  2022-02-20T05:00:00+08:00   
457  2022-05-23T05:00:00+08:00   
458  2022-05-23T05:00:00+08:00   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [111]:
# there are duplicates because of minor website naming differences

# convert published_date into a datetime column
df['published_date'] = pd.to_datetime(df['published_date'], errors='coerce')

# make sure content_chars is numeric (if it's not already)
df['content_chars'] = pd.to_numeric(df['content_chars'], errors='coerce')

# 1) sort so that for each title:
#    - most recent published_date comes first
#    - if dates tie, higher content_chars comes first
df = df.sort_values(
    ['published_date', 'content_chars'],
    ascending=[False, False]
)

# 2) drop duplicates on title, keeping the "best" row per title
df = df.drop_duplicates(subset='title', keep='first').reset_index(drop=True)

In [112]:
# Remove timezone information
df['published_date'] = df['published_date'].dt.tz_localize(None)
df.to_excel('Output/ST/merged_st_articles_2020_Jan_to_2025_Oct.xlsx', index=False)

# patch the scraped data

In [72]:
import pandas as pd
import re

In [92]:
df = pd.read_excel('Output/ST/merged_st_articles_2020_Jan_to_2025_Oct.xlsx')

In [93]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 985 entries, 0 to 984
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   title           985 non-null    object        
 1   url             985 non-null    object        
 2   published_date  985 non-null    datetime64[ns]
 3   content         985 non-null    object        
 4   content_chars   985 non-null    int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 38.6+ KB


In [94]:
# manually update an article url
# originally: https://www.straitstimes.com/singapore/housing/upgrading-works-slated-for-56000-hdb-flats
# to update to: https://www.straitstimes.com/singapore/housing/upgrading-works-start-for-56000-hdb-flats-built-between-1987-and-1997

df.loc[df['url'] == 'https://www.straitstimes.com/singapore/housing/upgrading-works-slated-for-56000-hdb-flats', 'url'] = 'https://www.straitstimes.com/singapore/housing/upgrading-works-start-for-56000-hdb-flats-built-between-1987-and-1997'
df.loc[df['url'] == 'https://www.straitstimes.com/singapore/housing/upgrading-works-start-for-56000-hdb-flats-built-between-1987-and-1997', 'published_date'] = pd.to_datetime("2022-02-20 13:00:00")


In [95]:
pd.set_option('display.max_colwidth', None)

print(df.loc[df['title'] == "BTO sales: Over 5,700 flats on offer", 'content'].to_string(index=False))


An artist's impression of ParkView @ Bidadari.\n\nAlready a subscriber? Log in\n\nGet full access to trusted news\n\nUnlock every story, analysis and exclusive from Singapore’s most established newsroom\n\nEnjoy these subscriber benefits\n\nFranklin Sports\n\n$49.83\n\n$76.92 35% off\n\nFossil\n\n$103.98\n\n$235.00 56% off\n\nMiffy\n\n$10.00\n\n$29.90 67% off\n\nOrico\n\n$21.00\n\n$40.00 48% off\n\nJo Malone\n\nFree Gifts\n\n$78.00\n\nPhilips\n\n$28.49\n\n$49.00 42% off\n\nAdidas\n\n$44.90\n\n$69.00 35% off\n\nHydroJug\n\n$29.84\n\n$56.07 47% off\n\nCornell\n\n$39.80\n\n$129.00 69% off\n\nVanish\n\n$15.00\n\n$19.10 21% off\n\nAkemi\n\n$16.07\n\n$38.00 58% off\n\n&honey\n\n$20.28\n\n$33.80 40% off\n\nMiniso\n\n$5.90\n\n$7.91 25% off\n\nHatchimals\n\n$12.78\n\n$31.81 60% off\n\nTiger\n\n$157.00\n\n$399.00 61% off


In [96]:
pattern = (
    r"(Shark\s*\$329\.00\s*\$649\.00\s*49% off)"
    r"|"
    r"(Franklin Sports\s*\$49\.83)"
)

mask = df['content'].str.contains(pattern, case=False, regex=True, na=False)

error_df = df.loc[mask]
len(error_df)


C:\Users\Wei'En\AppData\Local\Temp\ipykernel_36320\3053210677.py:7: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = df['content'].str.contains(pattern, case=False, regex=True, na=False)


8

In [97]:
error_df

,title,url,published_date,content,content_chars
423,BTO flats priced for various budgets,https://www.straitstimes.com/singapore/housing/housing-bto-flats-priced-for-various-budgets,2022-10-18 05:00:00,Shark\n\n$329.00\n\n$649.00 49% off\n\nWMF\n\n$23.99\n\n$49.00 51% off\n\nSkater\n\n$8.73\n\n$19.00 54% off\n\nTefal\n\n$78.99\n\n$169.00 53% off\n\nKate Spade\n\n$14.98\n\n$28.00 47% off\n\nUgreen\n\n$37.98\n\n$62.99 40% off\n\nDyson\n\n$379.00\n\n$549.00 31% off\n\nSukin\n\n$13.30\n\n$36.00 63% off\n\nOwala\n\n$30.11\n\n$48.00 37% off\n\nDuracell\n\n$11.40\n\n$19.01 40% off\n\nKipling\n\n$29.98\n\n$40.43 26% off\n\nBodum\n\n$19.98\n\n$36.80 46% off\n\nLego\n\n$11.90\n\n$14.90 20% off\n\nUSAopoly\n\n$33.63\n\n$66.00 49% off\n\nCharlie Mackesy\n\n$23.28\n\n$59.00 61% off\n\nNew Subscriber Exclusive\n\nUnwrap more than just news\n\nSubscribe to ST All-Digital + Print for $29.90/month and enjoy $180 off your Takashimaya shopping\n\n24-month contract | T&Cs apply,674
432,Smaller flats' resale prices tracked,https://www.straitstimes.com/singapore/housing/hdb-smaller-flats-resale-prices-tracked,2022-10-09 05:00:00,Shark\n\n$329.00\n\n$649.00 49% off\n\nWMF\n\n$23.99\n\n$49.00 51% off\n\nSkater\n\n$8.73\n\n$19.00 54% off\n\nTefal\n\n$78.99\n\n$169.00 53% off\n\nKate Spade\n\n$14.98\n\n$28.00 47% off\n\nUgreen\n\n$37.98\n\n$62.99 40% off\n\nDyson\n\n$379.00\n\n$549.00 31% off\n\nSukin\n\n$13.30\n\n$36.00 63% off\n\nOwala\n\n$30.11\n\n$48.00 37% off\n\nDuracell\n\n$11.40\n\n$19.01 40% off\n\nKipling\n\n$29.98\n\n$40.43 26% off\n\nBodum\n\n$19.98\n\n$36.80 46% off\n\nLego\n\n$11.90\n\n$14.90 20% off\n\nUSAopoly\n\n$33.63\n\n$66.00 49% off\n\nCharlie Mackesy\n\n$23.28\n\n$59.00 61% off\n\nNew Subscriber Exclusive\n\nUnwrap more than just news\n\nSubscribe to ST All-Digital + Print for $29.90/month and enjoy $180 off your Takashimaya shopping\n\n24-month contract | T&Cs apply,674
457,266 million-dollar resales so far this year,https://www.straitstimes.com/singapore/hdb-266-million-dollar-resales-so-far-this-year,2022-09-26 05:00:00,Shark\n\n$329.00\n\n$649.00 49% off\n\nWMF\n\n$23.99\n\n$49.00 51% off\n\nSkater\n\n$8.73\n\n$19.00 54% off\n\nTefal\n\n$78.99\n\n$169.00 53% off\n\nKate Spade\n\n$14.98\n\n$28.00 47% off\n\nUgreen\n\n$37.98\n\n$62.99 40% off\n\nDyson\n\n$379.00\n\n$549.00 31% off\n\nSukin\n\n$13.30\n\n$36.00 63% off\n\nOwala\n\n$30.11\n\n$48.00 37% off\n\nDuracell\n\n$11.40\n\n$19.01 40% off\n\nKipling\n\n$29.98\n\n$40.43 26% off\n\nBodum\n\n$19.98\n\n$36.80 46% off\n\nLego\n\n$11.90\n\n$14.90 20% off\n\nUSAopoly\n\n$33.63\n\n$66.00 49% off\n\nCharlie Mackesy\n\n$23.28\n\n$59.00 61% off\n\nNew Subscriber Exclusive\n\nUnwrap more than just news\n\nSubscribe to ST All-Digital + Print for $29.90/month and enjoy $180 off your Takashimaya shopping\n\n24-month contract | T&Cs apply,674
466,"HDB, condo rents rise again",https://www.straitstimes.com/singapore/housing/housing-hdb-condo-rents-rise-again,2022-09-15 05:00:00,Shark\n\n$329.00\n\n$649.00 49% off\n\nWMF\n\n$23.99\n\n$49.00 51% off\n\nSkater\n\n$8.73\n\n$19.00 54% off\n\nTefal\n\n$78.99\n\n$169.00 53% off\n\nKate Spade\n\n$14.98\n\n$28.00 47% off\n\nUgreen\n\n$37.98\n\n$62.99 40% off\n\nDyson\n\n$379.00\n\n$549.00 31% off\n\nSukin\n\n$13.30\n\n$36.00 63% off\n\nOwala\n\n$30.11\n\n$48.00 37% off\n\nDuracell\n\n$11.40\n\n$19.01 40% off\n\nKipling\n\n$29.98\n\n$40.43 26% off\n\nBodum\n\n$19.98\n\n$36.80 46% off\n\nLego\n\n$11.90\n\n$14.90 20% off\n\nUSAopoly\n\n$33.63\n\n$66.00 49% off\n\nCharlie Mackesy\n\n$23.28\n\n$59.00 61% off\n\nNew Subscriber Exclusive\n\nUnwrap more than just news\n\nSubscribe to ST All-Digital + Print for $29.90/month and enjoy $180 off your Takashimaya shopping\n\n24-month contract | T&Cs apply,674
520,Revised occupation period rule,https://www.straitstimes.com/singapore/housing/sers-%E2%80%A2-revised-occupation-period-rule,2022-06-18 05:00:00,Shark\n\n$329.00\n\n$649.00 49% off\n\nWMF\n\n$23.99\n\n$49.00 51% off\n\nSkater\n\n$8.73\n\n$19.00 54% off\n\nT

In [98]:
# function to rescrape article

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

def init_browser():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=chrome_options)

    # Open ST homepage for login
    LOGIN_URL = "https://www.straitstimes.com/"
    driver.get(LOGIN_URL)

    print("🔐 Please login manually in the Chrome window...")
    input("➡️  Press ENTER here after you have successfully logged in...")

    print("✅ Login detected. Browser session ready.")
    return driver

# Initialize once
driver = init_browser()

from bs4 import BeautifulSoup
import pandas as pd
import time

def patch_st_article_content(df, url, driver, content_column="content"):
    """
    Re-scrape a Straits Times article using Selenium (to bypass paywall)
    and update dataframe in-place.
    """

    print(f"\nRe-scraping: {url}")

    try:
        driver.get(url)
        time.sleep(2)  # allow page to load fully after paywall gate
        html = driver.page_source

    except Exception as e:
        print("❌ Selenium error while loading page:", e)
        return None

    soup = BeautifulSoup(html, "lxml")

    # Extract article wrapper
    wrapper = soup.select_one("div.storyline-wrapper.default")

    if wrapper is None:
        print("❌ storyline-wrapper not found. Page may still be behind paywall.")
        return None

    # Extract all paragraphs inside archival-wrapper
    paragraphs = [
        div.get_text(" ", strip=True)
        for div in wrapper.select("div.archival-wrapper")
    ]

    if not paragraphs:
        print("❌ No archival-wrapper text found.")
        return None

    scraped_content = "\n\n".join(paragraphs)

    # Update df
    mask = df["url"] == url
    updated_rows = mask.sum()

    if updated_rows > 0:
        df.loc[mask, content_column] = scraped_content
        print(f"✅ Updated {updated_rows} row(s).")
    else:
        print("⚠️ URL not found in dataframe.")

    return scraped_content


🔐 Please login manually in the Chrome window...


➡️  Press ENTER here after you have successfully logged in... 


✅ Login detected. Browser session ready.


In [99]:
urls_to_patch = error_df['url'].dropna().unique()

for url in urls_to_patch:
    patch_st_article_content(df, url, driver=driver)
    time.sleep(SLEEP_BETWEEN_ARTICLES)



Re-scraping: https://www.straitstimes.com/singapore/housing/housing-bto-flats-priced-for-various-budgets
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/hdb-smaller-flats-resale-prices-tracked
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/hdb-266-million-dollar-resales-so-far-this-year
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/housing-hdb-condo-rents-rise-again
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/sers-%E2%80%A2-revised-occupation-period-rule
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/upgrading-works-start-for-56000-hdb-flats-built-between-1987-and-1997
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/minister-calls-for-prudence
✅ Updated 1 row(s).

Re-scraping: https://www.straitstimes.com/singapore/housing/over-5700-flats-on-offer
✅ Updated 1 row(s).


In [102]:
df.to_excel('Output/ST/final_st_articles_2020_Jan_to_2025_Oct.xlsx', index=False)

# clean scraped data

In [29]:
import pandas as pd
import re

In [30]:
df = pd.read_excel('Output/ST/final_st_articles_2020_Jan_to_2025_Oct.xlsx')

In [39]:
pd.set_option('display.max_colwidth', None)
print(df.loc[df['title'] == "Water tank not tampered with, but probe ongoing", 'content'].to_string(index=False))

The water tank on an Ang Mo Kio block where the tap water was found to have trace amounts of hydrocarbon - the kind of compounds used in anti-mosquito oil - was not tampered with.\n\nBut investigations are continuing to determine how the hydrocarbons got into the tank located on the rooftop, Ang Mo Kio Town Council (AMKTC) said yesterday.\n\nThe tank is connected to a ground-floor drain where anti-mosquito oiling is done, via a discharge pipe.\n\nThe town council will work with the authorities to review the piping system to the water tank, its spokesman said, without elaborating.\n\nIt has also submitted a sample of its anti-mosquito oil to national water agency PUB for testing to see if the constituents match those of the hydrocarbons found by the agency.\n\nThe spokesman added: "Upon being alerted to (the issue) on Jan 13, we immediately checked the locks to the rooftop access and the water tanks and these were not tampered with.\n\n"We have already conducted thorough inspections on 

In [32]:
# remove irrelevant phrases

lines_to_remove = ["Swipe. Select. Stay informed.",
    "Singapore\n\n",
    "World\n\n",
    "Business\n\n",
    "\n\nBusiness",
    "Asia\n\n",
    "Life\n\n",
    "Already a subscriber? Log in\n\nGet full access to trusted news\n\nUnlock every story, analysis and exclusive from Singapore’s most established newsroom\n\nEnjoy these subscriber benefits",
    "SEE THE BIG STORY", "SEE TOP OF THE NEWS", "SEE HOME",
]

pattern = "|".join(re.escape(line) for line in lines_to_remove)

df['content'] = (
    df['content']
    .str.replace(pattern, "", regex=True)
    #.str.replace(r"\s+", " ", regex=True)  # clean extra spaces
    .str.strip()
)

# remove "SINGAPORE – " or "SINGAPORE - " at the START of the string
df['content'] = df['content'].str.replace("SINGAPORE ( THE NEW PAPER ) - ", '', regex=True)
df['content'] = df['content'].str.replace("SINGAPORE ( THE BUSINESS TIMES ) - ", '', regex=True)
df['content'] = df['content'].str.replace("SINGAPORE (BLOOMBERG) - ", '', regex=True)
df['content'] = df['content'].str.replace(r'^(SINGAPORE\s*[–-]\s*)', '', regex=True)


In [42]:
# all articles are of Singapore context

In [35]:
df.to_excel('Output/ST/cleaned_st_articles_2020_Jan_to_2025_Oct.xlsx', index=False)